## Cuaderno de creación de lista negra de usuarios

El código definido a continuación tiene como objetivo analizar las publicaciones de la carpeta [`datos`](/datos/), y crear una lista de usuarios que no deberían ser considerados en el análisis de datos.

Se consideraran para esta lista usuarios que sean autores de: 

* Publicaciones hechas para fomentar el cuidado de la salud mental
    * Si bien se pueden encontrar palabras clave que indiquen tendencias suicidas, no necesariamente reflejan problemas mentales en quien las publica
* Publicaciones repetidas (spam o bots)

Para ejecutar las siguientes instrucciones, se empleará el lenguaje de R.

Será de utilidad en el análisis, además, el paquete `tm`, que permite realizar análisis sobre texto.

In [1]:
# Instalar paquetes necesarios
install.packages(
    c(
        "tm",
        "dplyr",
        "stringr",
        "SnowballC"
    ),
    repos = "http://cran.us.r-project.org"
)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages(c("tm", "dplyr", "stringr", "SnowballC"), repos = "http://cran.us.r-project.org"):
“'lib = "/usr/local/lib/R/site-library"' is not writable”


ERROR: Error in install.packages(c("tm", "dplyr", "stringr", "SnowballC"), repos = "http://cran.us.r-project.org"): unable to install packages


In [3]:
# Se cargan las librerías
library('tm')
library('dplyr')
library('SnowballC')

Warning message:
"package 'tm' was built under R version 4.2.3"
Loading required package: NLP



Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'SnowballC' was built under R version 4.2.3"


In [4]:
# Se carga el archivo de texto
publicaciones_ahorcarme <- read.csv("./datos/ahorcarme_complete.csv")
publicaciones_colgarme <- read.csv("./datos/colgarme_complete.csv")

In [5]:
head(publicaciones_ahorcarme)

,User,Content,Date,URL,Coordinates,Place
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,nadaespecial__,Con la misma soga puedo ahorcarme o saltar,2014-12-30 22:55:34,https://twitter.com/nadaespecial__/status/550062666434506752,NA,NA
2,LooreenaaN,"""@SoyBuenCatre: Debí ahorcarme con el cordón umbilical.""",2014-12-30 22:04:21,https://twitter.com/LooreenaaN/status/550049781054439424,NA,NA
3,errorbizarro_,"Estoy harto de ser pobre, asique ayer intente ahorcarme, pero en el intento el techo se me cayó encima, osea que también soy gordo. :(",2014-12-30 21:57:08,https://twitter.com/errorbizarro_/status/550047965302513664,NA,NA
4,Straatoo,"Hay un viejo culiao acá que tiene un humor culiao tan ahueonao, que me dan ganas de tirarme ahorcarme de una wea",2014-12-30 21:29:24,https://twitter.com/Straatoo/status/550040983464083457,NA,NA
5,yadizrh,@emilioelde yo sería mas original..... Soy capaz de ahorcarme con un fideo cosido si no me contestas guapetón. I love you!!! ^_^,2014-12-30 19:27:45,https://twitter.com/yadizrh/status/550010368035405826,NA,NA
6,AlexanndraMn,Tengo el pelo tan largo que podría ahorcarme con él.,2014-12-30 19:23:19,https://twitter.com/AlexanndraMn/status/550009254745083905,NA,NA


In [3]:
# Obtenemos del dataframe las que tengan usuarios que aparezcan más de N veces

obtener_publicaciones <- function(dataframe, pub_minimas) {

    dataframe <- dataframe %>% 
        group_by(User) %>% 
        filter(n() > pub_minimas)

    return(dataframe)
}

In [4]:
# Filtramos los usuarios que tengan más de 5 publicaciones
publicaciones_ahorcarme_reducidas <- obtener_publicaciones(publicaciones_ahorcarme, 5)
publicaciones_colgarme_reducidas <- obtener_publicaciones(publicaciones_colgarme, 5)

In [5]:
# Se juntan los vectores que contienen los nombres de los usuarios, para ser unificados en una sola lista negra
# Se le suministrarán varios vectores de usuarios, y se devolverá un vector con los usuarios únicos

juntar_usuarios <- function(listas_usuarios) {

    usuarios <- c()

    for (lista in listas_usuarios) {
        usuarios <- c(usuarios, lista)
    }

    usuarios <- unique(usuarios)

    return(usuarios)
}

In [6]:
# Creamos la lista de los usuarios que aparecen más de 5 veces en las publicaciones

usuarios_ahorcarme <- publicaciones_ahorcarme_reducidas$User
usuarios_colgarme <- publicaciones_colgarme_reducidas$User

lista_negra <- juntar_usuarios(
    c(
        usuarios_ahorcarme,
        usuarios_colgarme
    )
)

In [9]:
head(lista_negra)

[1] "soiiviic"      "gatoacostadou" "Lucasnun_"     "cigarettommy" 
[5] "lwtironic"     "CyanideMoon"

Ahora, podemos revisar manualmente los tweets de los usuarios que colocamos en la lista negra bajo este criterio, para determinar si realmente sus publicaciones son despreciables para el análisis.

In [7]:
# Obtenemos los tweets de estos usuarios, para revisarlos manualmente

# Ahorcarme
tweets_lisneg_ahorcarme <- publicaciones_ahorcarme_reducidas %>%
    filter(User %in% lista_negra)

# Colgarme
tweets_lisneg_colgarme <- publicaciones_colgarme_reducidas %>%
    filter(User %in% lista_negra)

In [1]:
escribir_dataframe <- function(dataframe, ruta_archivo) {
    write.table(
        dataframe,
        file = ruta_archivo,
        row.names = FALSE,
        col.names = TRUE,
        quote = TRUE,
        sep = ","
    )
}

In [11]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(tweets_lisneg_ahorcarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_ahorcarme.csv")
# Colgarme
escribir_dataframe(tweets_lisneg_colgarme, "./datos_limpios/datos_por_revisar/tweets_lisneg_colgarme.csv")

Luego de revisar los tweets, encontré un fragmento que hace alusión a una connotación sexual de la palabra concurrente. Por lo tanto, aquellos usuarios cuyas publicaciones que contienen la palabra "ahorcarme" serán añadidos a la lista negra.

Es importante considerar que, si bien emplean la palabra bajo un concepto sexual, puede no ser la única forma en la que la usan; de hecho, logré encontrar varios perfiles en los que hacen un uso sexual y violento del verbo. De tal manera, de este grupo de usuarios que tergiversan de esta forma la acción, únicamente se seleccionarán aquellos que emplean la palabra exclusivamente en un contexto sexual, es decir, en todos los tweets que les fueron recolectados que les pertenecen.

In [5]:
# Función que obtiene los tweets de un usuario y determina si todos contienen una serie de textos claves

tweetsSonRepetitivos <- function(dataset, usuario, textosClave) {
    # Obtenemos los tweets del usuario
    tweets <- dataset %>%
        filter(User == usuario) %>%
        pull(Content) # pull() devuelve un vector con los valores de la columna
    
    # Revisamos si todos los tweets contienen alguno de los textos clave
    for (texto in textosClave){
        todosContienen <- all(grepl(texto, tweets, ignore.case = TRUE))

        # Nos conformamos con saber que uno de los textos que deseamos excluir está en los tweets,
        # pues es suficiente para determinar que el usuario repite contenido
        if (todosContienen) {
            return (TRUE)
        }
    }
    
    return (todosContienen)
}

In [6]:
# Función que toma un dataset y una serie de textos clave, y devuelve los usuarios que tienen todos
# sus tweets con al menos uno de esos textos clave

obtenerUsuariosRepetitivos <- function(dataset, textosClave) {
    # Obtenemos los usuarios
    usuarios <- dataset %>%
        distinct(User) %>%
        pull(User)
        
    # Obtenemos los usuarios que tienen todos sus tweets con alguno de sus textos clave en este vector
    usuarios_repetitivos <- c()
    
    for (usuario in usuarios) {

        if (tweetsSonRepetitivos(dataset, usuario, textosClave)) {
            usuarios_repetitivos <- c(usuarios_repetitivos, usuario)
            break
        }
    }
    
    return (unique(usuarios_repetitivos))
}

In [14]:
# Fragmento
frags_frase = c(
    "ahorcame por dios ahorcame hasta el punto de que tus manos"
)

# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados, siempre y cuando todos los tweets del usuario contengan la frase

usuarios_frags_ahorcarme <- obtenerUsuariosRepetitivos(publicaciones_ahorcarme, frags_frase)

Otro son fragmentos de una canción, en los tweets con la palabra clave "colgarme"

Los fragmentos son los siguientes:

<blockquote>
    <ul>
        <li> Niña, dame una pestaña de tus ojos para colgarme de amor por ti </li>
        <li> Colgarme de cualquiera que le gusta trasnochar </li>
    </ul>
</blockquote>

In [16]:
# Fragmentos
frags_frase = c(
    "Niña, dame una pestaña de tus ojos para colgarme de amor por ti",
    "Colgarme de cualquiera que le gusta trasnochar"
)
# Localizar los tweets que contengan la frase, y de ellos tomar los usuarios, para agregarlos a los que ya
# estaban listados.

usuarios_frags_colgarme <- obtenerUsuariosRepetitivos(publicaciones_colgarme, frags_frase)

Igualmente, se removerá cualquier uso de la llamada en el contexto de una conversación telefónica.

Por su parte, otros usos de la palabra que se refieran a una acción como "depender de" o "aprovecharse de" podrían requerir de otro tipo de servicios para ser procesados.

En [este notebook](./filtrado_tweets_ahorcarme.ipynb) estaré trabajando en otros usos de la palabra a través de los servicios que ofrece Microsoft Azure. Indagaré en el uso de los servicios cognitivos o las integraciones con OpenAI para proveer un filtro de las publicaciones con un procesamiento del lenguaje más intuitivo y preciso.

In [7]:
raiz_verbo_aparece <- function(texto, raices_a_comparar, language) {

    palabras <- unlist(strsplit(texto, " "))
    raices_palabras <- wordStem(palabras, language = language)
    
    for (raiz_palabra in raices_palabras) {
        
        for (raiz_verbo in raices_a_comparar) {
            
            if ( grepl(raiz_verbo, raiz_palabra, ignore.case = TRUE) ) {
                return(TRUE)
            }
        }

    }

    return(FALSE)

}

In [8]:
# Usando el paquete tm, se buscarán los tweets que contengan alguna conjugación de los verbos indicados

excluir_tweets_segun_palabras <- function(dataframe, palabras, language) {

    # Se obtienen las raíces de las palabras
    raices_palabras <- wordStem(palabras, language = language)

    # Se obtienen los tweets que no contengan ninguna de las raíces de las palabras (i.e., el verbo no aparece
    # en ninguna conjugación)
    tweets_sin_palabras <- dataframe %>%
        mutate(
            contiene_raices_palabras = sapply(Content, raiz_verbo_aparece, raices_palabras, language)
        ) %>%
        filter(contiene_raices_palabras == FALSE) %>%
        select(-contiene_raices_palabras) # Se elimina la columna auxiliar

    return(tweets_sin_palabras)
}

In [9]:
# No quitamos al usuario completamente porque el uso de la palabra "colgar" en una llamada no le excluye de usarla con
# intenciones autolesivas, por lo que nos limitamos a excluir esas publicaciones

palabras_contexto_telefonico <- c(
    "llamar",
    "marcar",
    "responder",
    "contestar"
)

# Demostrando el uso del paquete tm
raices_palabras_contexto_telefonico <- wordStem(palabras_contexto_telefonico, language = "spanish")
print(raices_palabras_contexto_telefonico)

[1] "llam"    "marc"    "respond" "contest"


In [28]:
# Eliminamos los tweets que contengan alguna conjugación de los verbos que indican que se habla de una llamada

tweets_colgarme_limpios <- excluir_tweets_segun_palabras(
    publicaciones_colgarme,
    palabras_contexto_telefonico,
    "spanish"
)

In [29]:
# Escribir tweets para no depender de la sesión de R
escribir_dataframe(tweets_colgarme_limpios, "./datos_limpios/datos_por_revisar/tweets_colgarme_limpios.csv")

In [36]:
# Escribir los mismos tweets, pero extrayendo aquellos cuyos usuarios aparecen con frecuencia

write.table(
    obtener_publicaciones(tweets_colgarme_limpios, 5),
    file = "./datos_limpios/datos_por_revisar/tweets_colgarme_radicados_5_ocurrencias.csv",
    row.names = FALSE,
    col.names = TRUE,
    quote = TRUE,
    sep = ","
)

In [31]:
# Agregar los usuarios a la lista de usuarios

lista_negra <- juntar_usuarios(
    c(
        lista_negra,
        usuarios_frags_ahorcarme,
        usuarios_frags_colgarme
    )
)

In [30]:
# Escribimos las listas de usuarios para no depender de la sesión de R
writeLines(usuarios_frags_ahorcarme, "./datos_limpios/datos_por_revisar/usuarios_frag_ahorcarme.txt")
writeLines(usuarios_frags_colgarme, "./datos_limpios/datos_por_revisar/usuarios_frag_colgarme.txt")

In [32]:
# Escribir la lista negra en un archivo
writeLines(
    lista_negra,
    file = "./datos_limpios/datos_por_revisar/lista_negra.txt",
)

A partir de aquí, una vez que hallamos revisado la lista negra de usuarios, vamos a limpiar los conjuntos de datos para obtener únicamente la información útil.

In [33]:
# Excluir a estos usuarios de los datasets, pero a partir de la lista negra de usuarios ya revisada

# Leer lista
lista_negra_revisada <- readLines("./datos_limpios/lista_negra_revisada.txt", encoding = "UTF-8")

In [35]:
# Reescribir lista negra, para obtener sólo valores únicos
lista_negra_revisada <- unique(lista_negra_revisada) %>% sort()
writeLines(lista_negra_revisada, "./datos_limpios/lista_negra_revisada.txt")

In [ ]:
retirar_tweets_usuarios <~ function(dataframe, usuarios) {
    return (
        dataframe %>%
            filter(!User %in% usuarios)
    )
}

In [ ]:
# Ahorcarme
publicaciones_ahorcarme <- retirar_tweets_usuarios(publicaciones_ahorcarme, lista_negra_revisada)

# Colgarme
publicaciones_colgarme <- retirar_tweets_usuarios(tweets_colgarme_limpios, lista_negra_revisada)

In [ ]:
# Guardamos los tweets en un archivo

# Ahorcarme
escribir_dataframe(publicaciones_ahorcarme, "./datos_limpios/ahorcarme_filtered.csv")

# Colgarme
escribir_dataframe(publicaciones_colgarme, "./datos_limpios/colgarme_filtered.csv")